In [1]:
import sys
import os
import random
# Counter
from collections import Counter
# Package
import somhos.resources.dataset as rd
import somhos.resources.queries as rq
from somhos.methods.useful import save_pickle, load_pickle, wordvectors_centroid
from somhos.config.paths import *

Default path

In [2]:
prefix_path = "../../"
data_path = get_relative_path(prefix_path, V9GAMMA_PATH)
os.path.exists(data_path)

True

Load samples

In [3]:
# Load sample A
docs_sample_a_path = get_relative_path(data_path, DOCS_SAMPLE_A_SUFFIX)
docs_sample_a = load_pickle(docs_sample_a_path)
# Load sample B
docs_sample_b_path = get_relative_path(data_path, DOCS_SAMPLE_B_SUFFIX)
docs_sample_b = load_pickle(docs_sample_b_path)

print("Samples size: (%d, %d)" % (len(docs_sample_a), len(docs_sample_b)))

Samples size: (5000, 5000)


Load samples content

In [4]:
docs_samples_content_path = get_relative_path(data_path, DOCS_SAMPLES_CONTENT)
samples_content = load_pickle(docs_samples_content_path)
print("Docs: %d" % len(samples_content))

Docs: 10000


In [5]:
from itertools import product
import numpy as np
from scipy.spatial.distance import cosine as cosine_distance

Load counts documents by keyprase

In [6]:
docs_samples_kps_doc_count_path = get_relative_path(data_path, DOCS_SAMPLES_KPS_DOC_COUNT)
samples_kps_in_docs_count = load_pickle(docs_samples_kps_doc_count_path)

Measure $idf(t, D) = log\frac{N}{\vert \left\{ d\in D:t\in d \right\} \vert}$

In [7]:
n_docs = len(docs_sample_a) + len(docs_sample_b)
t_idf = {t: np.log(n_docs/v) for t, v in samples_kps_in_docs_count.items()}

In [8]:
sample_a_indices = range(0, len(docs_sample_a))
sample_b_indices = range(0, len(docs_sample_b))

def get_content_for(index, samples_content, field='kps-normalized'):
    return samples_content[index][field]

def tf_doc(terms):
    tf = Counter(terms)
    cnt = sum(tf.values())
    return {k: v/cnt for k, v in tf.items()}

m_measure_sim = np.zeros([len(docs_sample_a), len(docs_sample_b)]) 
for i, j in product(sample_a_indices, sample_a_indices):
    # Term frequencies by document
    tf_a = tf_doc(get_content_for(docs_sample_a[i], samples_content))
    tf_b = tf_doc(get_content_for(docs_sample_b[j], samples_content))
    # Term list
    t_list = list(set(tf_a) | set(tf_b))
    # TF-IDF
    tf_idf_a = np.zeros([len(t_list),])
    tf_idf_b = np.zeros([len(t_list),])
    for pos, t in enumerate(t_list):
        if t in tf_a:
            tf_idf_a[pos] = tf_a[t] * t_idf[t]
        if t in tf_b:
            tf_idf_b[pos] = tf_b[t] * t_idf[t]
    # Measure
    measure_sim = 1.0 - cosine_distance(tf_idf_a, tf_idf_b)
    m_measure_sim[i, j] = measure_sim

print(m_measure_sim)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.00350385 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.00314453 0.         0.        ]]


In [9]:
docs_samples_measure_sim_path = get_relative_path(data_path, DOCS_SAMPLES_TF_IDF_KPS)
save_pickle(m_measure_sim, docs_samples_measure_sim_path)

In [10]:
n_singularvalues = 1
U, s, V = np.linalg.svd(m_measure_sim, full_matrices=False)
D = np.diag(s[:n_singularvalues])
m_udv_measure_sim = np.dot(U[:,:n_singularvalues], np.dot(D, V[:n_singularvalues,:]))
print(m_udv_measure_sim)

[[0.00062952 0.00072345 0.00349406 ... 0.00210431 0.00073329 0.00148477]
 [0.00043288 0.00049747 0.00240263 ... 0.001447   0.00050423 0.00102098]
 [0.00010388 0.00011938 0.00057656 ... 0.00034724 0.000121   0.000245  ]
 ...
 [0.00012329 0.00014168 0.00068428 ... 0.00041211 0.00014361 0.00029078]
 [0.00089562 0.00102925 0.00497097 ... 0.00299379 0.00104325 0.00211237]
 [0.00024734 0.00028425 0.00137284 ... 0.0008268  0.00028811 0.00058338]]


In [11]:
docs_samples_measure_sim_udv_path = get_relative_path(data_path, DOCS_SAMPLES_TF_IDF_UDV_KPS)
save_pickle(m_udv_measure_sim, docs_samples_measure_sim_udv_path)